# Introducción al Aprendizaje Automático &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; UANL - FCFM
---
## Sistema de recomendacion de peliculas a partir de un modelo de similaridad

**Alumnos**
- Ian Mauricio Saucedo Alemán &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 1868954
- Jared Evander Fuentes Sánchez &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;1868002
- Vicente Garza Reyna	&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;1847176
- Eduardo Alan Hernandez Villasana &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;1941416
- Sebastián Terrazas Santillana &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;1847317

### Lectura y formato de datos

Importamos las librerias principales a utilizar, y la conexion a drive por cuestion de facilidad al momento de acceder a los datos

In [ ]:
import pandas as pd 
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

PATH_DATASETS = '/content/drive/MyDrive/PIA_Datasets/API Datasets/'

In [ ]:
movie_data_df = pd.read_csv(PATH_DATASETS + 'movie_data.csv')
movie_50k_df = pd.read_csv(PATH_DATASETS + 'most_50k_popular.csv')
cast_df = pd.read_csv(PATH_DATASETS + 'cast.csv')
directors_df = pd.read_csv(PATH_DATASETS + 'directors.csv')
credits_df = pd.read_csv(PATH_DATASETS + 'movie_credits.csv')
genres_df = pd.read_csv(PATH_DATASETS + 'genres.csv')

Estructura de nuestro dataset principal

In [ ]:
movie_50k_df.shape

(50000, 6)

In [ ]:
movie_50k_df.head()

,Unnamed: 0,Id,Name,Year,Popularity,Genres
0,304032,566525,Shang-Chi and the Legend of the Ten Rings,2021,10359.702,"[28, 12, 14]"
1,304033,370172,No Time to Die,2021,5130.748,"[12, 28, 53]"
2,304034,580489,Venom: Let There Be Carnage,2021,4144.219,"[878, 28, 12]"
3,304035,522402,Finch,2021,3642.038,"[878, 18, 12]"
4,304036,763164,Apex,2021,2915.735,"[28, 53, 878]"


Estructura de los datasets de informacion 

In [ ]:
credits_df.head()

,Unnamed: 0,Id,Cast,Directors
0,0,566525,"[1489211, 1337, 1625558, 2979464, 123701, 1620...","[134686, 1144604, 1740449, 2422821]"
1,1,370172,"[8784, 17838, 121529, 5469, 17064, 1360281, 20...",[87257]
2,2,580489,"[2524, 57755, 1812, 2038, 81316, 1115, 2141479...","[1333, 5545, 15335, 1184487]"
3,3,522402,"[31, 572541, 1457107, 1151637, 1912057, 3308163]",[114404]
4,4,763164,"[2203, 62, 77109, 203630, 63564, 1692990, 2725...",[2613318]


Realizamos el linkeo del dataset principal con el de creditos para obtener la informacion de cada pelicula

In [ ]:
movie_50k_df['Id'].dtype, credits_df['Id'].dtype

(dtype('int64'), dtype('int64'))

In [ ]:
movies_and_credits_df = pd.merge(movie_50k_df, credits_df[['Id', 'Cast', 'Directors']], on=['Id'], how='left')

In [ ]:
movies_and_credits_df.head()

,Unnamed: 0,Id,Name,Year,Popularity,Genres,Cast,Directors
0,304032,566525,Shang-Chi and the Legend of the Ten Rings,2021,10359.702,"[28, 12, 14]","[1489211, 1337, 1625558, 2979464, 123701, 1620...","[134686, 1144604, 1740449, 2422821]"
1,304033,370172,No Time to Die,2021,5130.748,"[12, 28, 53]","[8784, 17838, 121529, 5469, 17064, 1360281, 20...",[87257]
2,304034,580489,Venom: Let There Be Carnage,2021,4144.219,"[878, 28, 12]","[2524, 57755, 1812, 2038, 81316, 1115, 2141479...","[1333, 5545, 15335, 1184487]"
3,304035,522402,Finch,2021,3642.038,"[878, 18, 12]","[31, 572541, 1457107, 1151637, 1912057, 3308163]",[114404]
4,304036,763164,Apex,2021,2915.735,"[28, 53, 878]","[2203, 62, 77109, 203630, 63564, 1692990, 2725...",[2613318]


In [ ]:
movies_and_credits_df = movies_and_credits_df.astype({'Genres': 'string', 'Cast': 'string', 'Directors': 'string'})

In [ ]:
eval(movies_and_credits_df['Cast'].iloc[0])

[1489211,
 1337,
 1625558,
 2979464,
 123701,
 1620,
 57609,
 2282,
 1998541,
 1068877,
 3074780,
 3201062,
 3218942,
 3216792,
 1381186,
 2422160,
 10075,
 44059,
 1792920,
 1319469,
 20479,
 2344691,
 216782,
 1535058,
 1838225,
 165214,
 62421,
 1691275,
 2651594,
 1920116,
 23680,
 30082,
 3129,
 103,
 60073,
 3114744,
 2535732,
 1102421]

Aqui asignamos los verdaderos valores de las columnas Cast, Directors y Genres a cada pelicula

In [ ]:
for i in range(len(movies_and_credits_df)): 
  try: 
    this_cast = eval(str(movies_and_credits_df['Cast'].iloc[i]))
    this_cast = cast_df[cast_df['Id'].isin(this_cast)].head(10)
    this_cast = str(this_cast['Name'].values)
    movies_and_credits_df['Cast'].iloc[i] = this_cast
  except: 
    movies_and_credits_df['Cast'].iloc[i] = '[]'
  try:
    this_directors = eval(str(movies_and_credits_df['Directors'].iloc[i]))
    this_directors = directors_df[directors_df['Id'].isin(this_directors)].head()
    this_directors = str(this_directors['Name'].values)
    movies_and_credits_df['Directors'].iloc[i] = this_directors
  except: 
    movies_and_credits_df['Directors'].iloc[i] = '[]'
  try:
    this_genres = eval(str(movies_and_credits_df['Genres'].iloc[i]))
    this_genres = genres_df[genres_df['Id'].isin(this_genres)].head()
    this_genres = str(this_genres['name'].values)
    movies_and_credits_df['Genres'].iloc[i] = this_genres
  except: 
    movies_and_credits_df['Genres'].iloc[i] = '[]'

In [ ]:
movies_and_credits_df.head(2)

,Unnamed: 0,Id,Name,Year,Popularity,Genres,Cast,Directors
0,304032,566525,Shang-Chi and the Legend of the Ten Rings,2021,10359.702,['Action' 'Adventure' 'Fantasy'],['Tony Leung Chiu-wai' 'Awkwafina' 'Simu Liu' ...,['Destin Daniel Cretton']
1,304033,370172,No Time to Die,2021,5130.748,['Action' 'Adventure' 'Thriller'],['Ana de Armas' 'Léa Seydoux' 'Daniel Craig' '...,['Cary Joji Fukunaga']


In [ ]:
movies_and_credits_df.columns

Index(['Unnamed: 0', 'Id', 'Name', 'Year', 'Popularity', 'Genres', 'Cast',
       'Directors'],
      dtype='object')

Entonces, juntaremos estas columnas en una sola donde se almacenaran los valores principales de cada pelicula

In [ ]:
def combined_features(row): 
  combined_features = str(row['Genres']) + ' ' + str(row['Cast']) + ' ' + str(row['Directors']) + ' ' + str(row['Year'])
  return combined_features

In [ ]:
movies_and_credits_df['combined_features'] = movies_and_credits_df.apply(combined_features, axis=1)

In [ ]:
movies_and_credits_df['combined_features'] = movies_and_credits_df['combined_features'].astype(str)

In [ ]:
movies_and_credits_df['combined_features'] = movies_and_credits_df['combined_features'].str.replace('[', '')
movies_and_credits_df['combined_features'] = movies_and_credits_df['combined_features'].str.replace(']', '')
movies_and_credits_df['combined_features'] = movies_and_credits_df['combined_features'].str.replace('\'', '')
movies_and_credits_df['combined_features'] = movies_and_credits_df['combined_features'].str.replace('\n', '', regex=True)

In [ ]:
movies_and_credits_df.head()

,Unnamed: 0,Id,Name,Year,Popularity,Genres,Cast,Directors,combined_features
0,304032,566525,Shang-Chi and the Legend of the Ten Rings,2021,10359.702,['Action' 'Adventure' 'Fantasy'],['Tony Leung Chiu-wai' 'Awkwafina' 'Simu Liu' ...,['Destin Daniel Cretton'],Action Adventure Fantasy Tony Leung Chiu-wai A...
1,304033,370172,No Time to Die,2021,5130.748,['Action' 'Adventure' 'Thriller'],['Ana de Armas' 'Léa Seydoux' 'Daniel Craig' '...,['Cary Joji Fukunaga'],Action Adventure Thriller Ana de Armas Léa Sey...
2,304034,580489,Venom: Let There Be Carnage,2021,4144.219,['Action' 'Adventure' 'Science Fiction'],['Tom Holland' 'Tom Hardy' 'Woody Harrelson' '...,[],Action Adventure Science Fiction Tom Holland T...
3,304035,522402,Finch,2021,3642.038,['Adventure' 'Drama' 'Science Fiction'],['Tom Hanks' 'Lora Martinez-Cunningham' 'Caleb...,[],Adventure Drama Science Fiction Tom Hanks Lora...
4,304036,763164,Apex,2021,2915.735,['Action' 'Science Fiction' 'Thriller'],['Bruce Willis' 'Lochlyn Munro' 'Neal McDonoug...,[],Action Science Fiction Thriller Bruce Willis L...


Reducimos la dimensionalidad de nuestro dataset por cuestiones de costo computacional

In [ ]:
movies_and_credits_df = movies_and_credits_df.iloc[:30000]

In [ ]:
movies_and_credits_df.shape

(30000, 9)

### Aplicación del modelo

Para nuestro modelo utilizaremos CountVectorizer para aprender la informacion principal de nuestras peliculas y cosine_similarity para calcular la similaridad entre peliculas. 

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
cv = CountVectorizer()
count_matrix = cv.fit_transform(movies_and_credits_df['combined_features'])

In [ ]:
cosine_sim = cosine_similarity(count_matrix)

In [ ]:
cosine_sim.shape

(30000, 30000)

### Prueba de rendimiento

Para una pelicula al azar, tomamos su nombre e index para realizar una consulta en nuestra matriz de similaridad. 

In [ ]:
random_movie = movies_and_credits_df.sample(1)
random_movie['Name'] = random_movie['Name'].astype('string')

In [ ]:
random_movie

,Unnamed: 0,Id,Name,Year,Popularity,Genres,Cast,Directors,combined_features
23767,245154,345170,Frank and Cindy,2015,4.751,['Comedy' 'Drama'],['Oliver Platt' 'Rene Russo' 'Jane Levy' 'John...,[],Comedy Drama Oliver Platt Rene Russo Jane Levy...


In [ ]:
movie_user_likes = random_movie['Name'].values[0]
movie_index = movies_and_credits_df[movies_and_credits_df['Name'] == movie_user_likes].index.values[0]
similar_movies = list(enumerate(cosine_sim[movie_index]))
sorted_similar_movies = sorted(similar_movies, key=lambda x:x[1], reverse=True)

Obtenemos una lista de las peliculas con mayor similaridad a nuestra pelicula elegida

In [ ]:
sorted_similar_movies[:10]

[(23767, 0.9999999999999998),
 (11756, 0.4803844614152615),
 (15061, 0.4803844614152615),
 (15257, 0.4803844614152615),
 (15737, 0.4803844614152615),
 (21178, 0.4803844614152615),
 (26669, 0.4803844614152615),
 (27517, 0.4803844614152615),
 (27971, 0.4803844614152615),
 (28140, 0.4803844614152615)]

In [ ]:
i=0
for movie in sorted_similar_movies:
    print(movies_and_credits_df['Name'].iloc[movie[0]])
    i=i+1
    if i>5:
        break

Frank and Cindy
Truman
The Student and Mister Henri
One for the Road
Seoul Searching
The Morning After
